# Solving Single Model
This notebook demonstrates how to create and solve a single model.
One model contains the first-stage reinforcement decision and the second-stage
network scheduling decision. The attacks that happen are uncertain and 
therefore the second-stage network scheduling decision is stochastic, which 
makes the model a mixed-integer stochastic program.

In [75]:
using Pkg;
Pkg.activate("..");
Pkg.instantiate();

  Activating project at `~/dev/uni/amo-team-project`


To solve our model we use the commercial solver Gurobi, because its performance
is superior to the open source solvers. Our model, like many MILP models, gets
computationally intractable very fast. Therefore, we need the fastest solver we
can get.

In [76]:
import Gurobi
GRB_ENV = Gurobi.Env()

Set parameter Username
Academic license - for non-commercial use only - expires 2024-01-19


Gurobi.Env(Ptr{Nothing} @0x0000560c2738bf10, false, 0)

First, we read the source data, which consists of the network specification and
the demand data.  
The network specification contains busses, lines and generators.

In [77]:
include("../src/rwth_parser.jl")
network = read_network("../data/Scenario_2013.xlsx")
loads = read_loads("../data/Scenario_2013.xlsx")
print()

Here we limit the demand data to the month of January. This is not required in
any way, but it is closer to what we later do in the simulation.

In [66]:
include("../src/loads.jl")
local_loads = remove_non_data_rows(filter_month(loads, 1))
print()

Next we generate our scenarios. To do that we first need different scenarios for
the weather, more specifically the wind and the solar irradiation, and also
scenarios for the attacks that happen.  
We can configure the number of attack scenarios and weather scenarios we want to
generate, as well as the number of rocket attacks per scenario.  

We generate wind speed with the help of the Rayleigh distribution with common
parameters and solar irradiation with the help of the Beta distribution with
common parameters.

To generate attack scenarios we expect that busses with a large line capacity
connected to them are more likely to be attacked. Therefore we sample N distinct
busses from our network, with the probability of a bus being sampled being
proportional to the line capacity connected to it.

In [67]:
include("../src/scenarios/generation.jl")
attack_scenarios = generate_attack_scenarios(network, 10, 10000)
weather_scenarios = generate_weather_scenarios(10000)
print()

A model with a high number of scenarios becomes computationally intractable very
fast. Therefore we reduce the number of scenarios by clustering them.  
For weather and demand we use k-means clustering, for the attacks we use 
k-medoids clustering, because the attacks are discrete and k-means could
result in non-discrete cluster centers.
The resulting cluster centers will be our reduced scenarios and the number of original data points assigned to each center will be used as the probability of that scenario.

In [68]:
include("../src/scenarios/reduction.jl")
reduced_attack_scenarios = reduce_binary_scenarios(attack_scenarios, 3)
reduced_weather_scenarios = reduce_continous_scenarios(weather_scenarios, 2)
reduced_load_scenarios = reduce_continous_scenarios(local_loads, 2)
print()

Now, we have three datasets containing demand, attack and weather scenarios.
A scenario in the final model needs to contain values for each of them.  
Therefore in the next step we combine the three datasets into one dataset,
by taking the cartesian product of the three datasets. 

In [69]:
include("../src/scenarios/assemble.jl")
scenarios = cartesian_scenarios(
    reduced_load_scenarios,
    reduced_attack_scenarios,
    reduced_weather_scenarios,
)
print()

Until now, we had wind speed and solar irradiation as weather scenarios.
However, our model needs to have maximum capacities of the solar power plants
and wind turbines. We therefore convert the wind speed and solar irradiation
into maximum capacities.  
To do that we follow the approach of [Saber et al.](https://ietresearch.onlinelibrary.wiley.com/doi/full/10.1049/iet-gtd.2014.0040).
The approach they use and an approximation we do on top of that is described in `./01_1_approximation_renewables.ipynb`.

In [70]:
scenarios = translate_weather_to_capacity(scenarios, network.generators)
print()

Before we can finally build the model we need to bring the scenarios into an appriopriate and performance optimized data format.

In [71]:
scenario_dict = convert_df_to_scenarios(scenarios)
print("Number of scenarios: $(length(scenario_dict))")

Number of scenarios: 12

Next we build the model. For a detailed description of the model, read our paper or look at the model in `./model.jl`.

In [72]:
include("../src/model.jl")
dispatch_model = DispatchModel(network, scenario_dict, 10.0)
init_model!(dispatch_model)
print()

Finally, we solve the model.

In [73]:
t = solve!(dispatch_model)
print()

In [74]:
import JuMP
print("Objective value: $(JuMP.objective_value(dispatch_model.m))")

Objective value: 1744.2195476633603